In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
strategy = tf.distribute.experimental.CentralStorageStrategy ()
from tensorflow.keras.callbacks import TensorBoard
import time

name = "image_classification-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(name))
 
with strategy.scope():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()



INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 32)        128       
 chNormalization)                                                
                 

In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load and preprocess the data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Compile and train the model

model.fit(train_images, train_labels, epochs=10, batch_size=128, validation_split=0.2, callbacks=[tensorboard])


Epoch 1/10
313/313 [==============================] - 107s 333ms/step - loss: 1.9046 - accuracy: 0.3627 - val_loss: 4.2023 - val_accuracy: 0.1061
Epoch 2/10
313/313 [==============================] - 103s 329ms/step - loss: 1.3394 - accuracy: 0.5191 - val_loss: 1.2195 - val_accuracy: 0.5715
Epoch 3/10
313/313 [==============================] - 129s 414ms/step - loss: 1.1273 - accuracy: 0.5973 - val_loss: 1.4612 - val_accuracy: 0.5217
Epoch 4/10
313/313 [==============================] - 100s 320ms/step - loss: 1.0140 - accuracy: 0.6410 - val_loss: 0.8668 - val_accuracy: 0.6880
Epoch 5/10
313/313 [==============================] - 152s 486ms/step - loss: 0.9216 - accuracy: 0.6722 - val_loss: 1.1853 - val_accuracy: 0.5894
Epoch 6/10
313/313 [==============================] - 125s 397ms/step - loss: 0.8598 - accuracy: 0.6989 - val_loss: 0.7410 - val_accuracy: 0.7397
Epoch 7/10
313/313 [==============================] - 128s 408ms/step - loss: 0.7985 - accuracy: 0.7185 - val_loss: 0.8695 -

In [3]:

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


313/313 [==============================] - 9s 28ms/step - loss: 0.6084 - accuracy: 0.7890
Test accuracy: 0.7889999747276306
